<a href="https://colab.research.google.com/github/Drecor1798/Proyecto_coderHouse_ciencia_de_datos/blob/main/Untitled49.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Plataforma Inteligente para la Selección Óptima de Zonas de Cultivo de las 100 prinsipales ciudades sugun Forbes


#Agenda

1. Descripccion de proyecto
2. Preparación del entorno
3. Preparación de datos
4. Análisis exploratorio
   *   Análisis de las ciudades más calurosas.
   *   Análisis de las ciudades más frías.
   *   Analisis de las ciudades mas calurosas de las 100 ciudades 100 ciudades más importantes de Forbes
   *   Analisis de las ciudades mas frias de las 100 ciudades 100 ciudades más importantes de Forbes
5. Ciudades con mayor promedio de temperatura
6. Ciudades con menor promedio
7. Agricultura
9. Insights
10. Concluciones




#Descripccion de proyeto

##Problema a Resolver

La agricultura enfrenta riesgos crecientes debido a:

* Cambios climáticos impredecibles

* Decisiones inadecuadas en la elección de zonas de cultivo

* Pérdidas económicas por bajos rendimientos o fallas en las cosechas

Muchos agricultores y tomadores de decisiones siembran sin conocer con precisión si las condiciones agroclimáticas locales son óptimas para los cultivos seleccionados.

##Objetivo de proyecto
Desarrollar una herramienta basada en datos climáticos y geoespaciales que identifique las ciudades o regiones con las condiciones óptimas de temperatura, precipitación y radiación solar para el cultivo de especies agrícolas específicas.


##Preguntas
* ¿Cómo varían las diferentes variables de temperatura (máxima, mínima, media, aparente) a lo largo del año en las ciudades más calurosas?
* ¿Cual es el periodo de tiempo (meses) que tienen mayor y menor temperatura de las ciudades con mayor temperatura?
* ¿Qué meses tienen más días con temperaturas agradables (entre 20 y 30 grados) en las ciudades más calurosas?
* ¿Cómo se distribuye la diferencia entre la temperatura media y la sensación térmica (diferencia aparente) en estas ciudades?
* ¿Cómo se distribuye la diferencia entre la temperatura máxima y mínima (amplitud térmica) en estas ciudades?
* ¿Cuáles son las ciudades con las mejores condiciones agroclimáticas en general, considerando temperatura, precipitación, radiación solar y balance hídrico?
* ¿Cómo varían la temperatura media y la precipitación a lo largo del tiempo en las ciudades con mejor clima para la agricultura?
* ¿Que cosecha se puede cultivar en cada ciudad




#Preparacion del entorno

In [ ]:
!pip install -q kaggle

In [ ]:
! gdown --id 1XL-U81WlWTjmc5ByZVd3l63wARxECvaL

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

In [ ]:
!kaggle datasets download -d bwandowando/forbes-top-100-cities-weather-data2020-ytd

In [ ]:
!kaggle datasets download -d samuelotiattakorah/agriculture-crop-yield

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from prophet import Prophet
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Data handling and preprocessing
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Model selection and evaluation
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingGridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, mean_absolute_error, r2_score

# Classification models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Regression models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

# Visualization
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Miscellaneous
import pickle
import os
import joblib

In [ ]:
!unzip /content/forbes-top-100-cities-weather-data2020-ytd.zip
!ls /content/forbes-top-100-cities-weather-data2020-ytd.zip

In [ ]:
df1 = pd.read_csv('/content/daily_data.csv')
df1.info()

In [ ]:
pd.set_option('display.max_columns', None)
serie = (df1.isnull().sum()/df1.shape[0])*100
serie.sort_values(ascending=False)
serie = serie[serie>0]
serie

In [ ]:
df2 = pd.read_csv('/content/hourly_data.csv')
df2.info()

In [ ]:
pd.set_option('display.max_columns', None)
serie = (df2.isnull().sum()/df2.shape[0])*100
serie.sort_values(ascending=False)
serie = serie[serie>0]
serie

In [ ]:
df3 = pd.read_csv('/content/cities.csv')
df3.info()

In [ ]:
pd.set_option('display.max_columns', None)
serie = (df3.isnull().sum()/df3.shape[0])*100
serie.sort_values(ascending=False)
serie = serie[serie>0]
serie

In [ ]:
df4 = pd.read_csv('/content/agriculture-crop-yield.zip')
df4.info()

In [ ]:
pd.set_option('display.max_columns', None)
serie = (df4.isnull().sum()/df4.shape[0])*100
serie.sort_values(ascending=False)
serie = serie[serie>0]
serie

In [ ]:
paises2 = df1['city_name'].unique()

# Iterar sobre cada país
for city_name in paises2:
    print(f"\n===== {city_name} =====")
    df_grupo = df1[df1['city_name'] == city_name]
    df_pivot = df_grupo.set_index('datetime')[[ 'temperature_2m_max','temperature_2m_min','temperature_2m_mean','apparent_temperature_mean']]
    display(df_pivot)

In [ ]:
# Define a function to categorize the climate based on temperature
def classify_climate(row):
    # You can adjust these thresholds based on your definition of each category
    if row['temperature_2m_mean'] < 10:
        return 'Frío'
    elif 10 <= row['temperature_2m_mean'] < 20:
        return 'Agradable'
    else:
        return 'Caluroso'

# Apply the function to create the 'clima_categoria' column
df1['clima_categoria'] = df1.apply(classify_climate, axis=1)

# Display the value counts for the new column to check the distribution
display(df1['clima_categoria'].value_counts())

In [ ]:
import pandas as pd

# --- Asegurar que datetime es tipo fecha
df1['datetime'] = pd.to_datetime(df1['datetime'])

# --- Crear variables adicionales
df1['año'] = df1['datetime'].dt.year
df1['mes'] = df1['datetime'].dt.month
df1['dia'] = df1['datetime'].dt.day
df1['dia_semana'] = df1['datetime'].dt.day_name()
df1['semana'] = df1['datetime'].dt.isocalendar().week



# --- Crear variables de temperatura adicionales
df1['amplitud_termica'] = df1['temperature_2m_max'] - df1['temperature_2m_min']
df1['diferencia_aparente'] = df1['apparent_temperature_mean'] - df1['temperature_2m_mean']

# --- Mostrar datos por ciudad
ciudades = df1['city_name'].unique()

for city in ciudades:
    print(f"\n===== {city} =====")

    df_ciudades = df1[df1['city_name'] == city]

    # Asegurarse de que esté ordenado por fecha
    df_pivot = df_ciudades.set_index('datetime').sort_index()[[
        'temperature_2m_max',
        'temperature_2m_min',
        'temperature_2m_mean',
        'apparent_temperature_mean',
        'amplitud_termica',
        'diferencia_aparente',
        'año',
        'mes',
        'dia',
        'dia_semana',
        'semana',
        'clima_categoria'
    ]]

    # Mostrar las primeras filas
    display(df_pivot.head())



#Análisis exploratorio

##Análisis de las ciudades más calurosas.

In [ ]:
# Asegurar que datetime es tipo fecha
df1['datetime'] = pd.to_datetime(df1['datetime'])

# Agrupar por ciudad y calcular el promedio de temperatura máxima
promedio_por_ciudad_calor = df1.groupby('city_name')['temperature_2m_max'].mean()

# Obtener las 10 ciudades con mayor promedio
top_10_promedios_max = promedio_por_ciudad_calor.sort_values(ascending=False).head(10)

# Mostrar resultados
print("Top 10 ciudades con mayor promedio de temperatura máxima:")
print(top_10_promedios_max)




##Análisis de las ciudades más frías.

In [ ]:
# Asegurar que datetime es tipo fecha
df1['datetime'] = pd.to_datetime(df1['datetime'])

# Agrupar por ciudad y calcular el promedio de temperatura máxima
promedio_por_ciudad_frio = df1.groupby('city_name')['temperature_2m_min'].mean()

# Obtener las 10 ciudades con mayor promedio
top_10_promedios_min = promedio_por_ciudad_frio.sort_values(ascending=True).head(10)

# Mostrar resultados
print("Top 10 ciudades con mayor promedio de temperatura minimas:")
print(top_10_promedios_min)


##Analisis de las ciudades mas calurosas de las 100 ciudades 100 ciudades más importantes de Forbes


In [ ]:
df1['datetime'] = pd.to_datetime(df1['datetime'])

# Promedios por mes
# Use a list instead of a tuple to select multiple columns
df_mes = df1.groupby(['año', 'mes'])[['temperature_2m_min', 'temperature_2m_max']].mean().reset_index()

# Estimar estación por rangos
def determinar_estacion(row):
    tmin = row['temperature_2m_min']
    tmax = row['temperature_2m_max']
    if tmax >= 27:
        return 'Verano'
    elif tmin < 10:
        return 'Invierno'
    elif 10 <= tmin < 18 and tmax < 27:
        return 'Primavera'
    else:
        return 'Otoño'

df_mes['estacion_aproximada'] = df_mes.apply(determinar_estacion, axis=1)

In [ ]:

paises_deseados = top_10_promedios_max.index
df_filtrado = df1[df1['city_name'].isin(paises_deseados)]

# --- Mostrar datos por ciudad (solo dentro de los países filtrados)
ciudades = df_filtrado['city_name'].unique()

for city in ciudades:
    print(f"\n===== {city} =====")

    df_ciudad = df_filtrado[df_filtrado['city_name'] == city]

    # Asegurarse de que esté ordenado por fecha
    df_pivot_calor = df_ciudad.set_index('datetime').sort_index()[[
        'temperature_2m_max',
        'temperature_2m_min',
        'temperature_2m_mean',
        'apparent_temperature_max',
        'apparent_temperature_min',
        'amplitud_termica',
        'diferencia_aparente',
        'año',
        'mes',
        'dia',
        'dia_semana',
        'semana',
        'clima_categoria'
    ]]

    # Mostrar las primeras filas
    display(df_pivot_calor.head())

##Analisis de las ciudades mas frias de las 100 ciudades 100 ciudades más importantes de Forbes

In [ ]:

paises_deseados_frios = top_10_promedios_min.index
df_filtrado_frio = df1[df1['city_name'].isin(paises_deseados_frios)]

# --- Mostrar datos por ciudad (solo dentro de los países filtrados)
ciudades_frio = df_filtrado_frio['city_name'].unique()

for city in ciudades_frio:
    print(f"\n===== {city} =====")

    df_ciudad_fria = df_filtrado_frio[df_filtrado_frio['city_name'] == city]

    # Asegurarse de que esté ordenado por fecha
    df_pivot_frio = df_ciudad_fria.set_index('datetime').sort_index()[[
        'temperature_2m_max',
        'temperature_2m_min',
        'temperature_2m_mean',
        'apparent_temperature_mean',
        'amplitud_termica',
        'diferencia_aparente',
        'año',
        'mes',
        'dia',
        'dia_semana',
        'semana',
        'clima_categoria'
    ]]

    # Mostrar las primeras filas
    display(df_pivot_frio.head())

#Ciudades con mayor promedio de temperatura

##¿Cómo varían las diferentes variables de temperatura (máxima, mínima, media, aparente) a lo largo del año en las ciudades más calurosas?


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Filtrar por año
df_filtrado = df_filtrado[df_filtrado['año'].between(2020, 2025)]
# Obtener ciudades del dataframe filtrado
ciudades = df_filtrado['city_name'].unique()

# Iterar sobre cada ciudad
for city in ciudades:
    print(f"\n===== {city} =====")

    # Filtrar datos para la ciudad
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city]

    # Agrupar por año y calcular promedios
    promedio_anual_calor = df_ciudad.groupby('año')['temperature_2m_max'].mean()
    promedio_total_calor = promedio_anual_calor.mean()

    promedio_anual_frio = df_ciudad.groupby('año')['temperature_2m_min'].mean()
    promedio_total_frio = promedio_anual_frio.mean()

    promedio_anual_medio = df_ciudad.groupby('año')['temperature_2m_mean'].mean()
    promedio_total_medio = promedio_anual_medio.mean()

    promedio_anual_aparente = df_ciudad.groupby('año')['apparent_temperature_mean'].mean()
    promedio_total_aparente = promedio_anual_aparente.mean()

    promedio_anual_amplitud = df_ciudad.groupby('año')['amplitud_termica'].mean()
    promedio_total_amplitud = promedio_anual_amplitud.mean()

    fig_subplots = make_subplots(
        rows=2, cols=3,
        subplot_titles=[
            "Promedio de calor por año",
            "Promedio de frío por año",
            "Promedio de temperatura media por año",
            "Temperatura aparente por año",
            "Amplitud térmica por año"
        ]
    )

    fig_subplots.add_trace(go.Bar(x=promedio_anual_calor.index, y=promedio_anual_calor, name="Temp. Máxima"), row=1, col=1)
    fig_subplots.add_trace(go.Bar(x=promedio_anual_frio.index, y=promedio_anual_frio, name="Temp. Mínima"), row=1, col=2)
    fig_subplots.add_trace(go.Bar(x=promedio_anual_medio.index, y=promedio_anual_medio, name="Temp. Media"), row=1, col=3)
    fig_subplots.add_trace(go.Bar(x=promedio_anual_aparente.index, y=promedio_anual_aparente, name="Temp. Aparente"), row=2, col=1)
    fig_subplots.add_trace(go.Bar(x=promedio_anual_amplitud.index, y=promedio_anual_amplitud, name="Amplitud Térmica"), row=2, col=2)

    fig_subplots.update_layout(height=700, width=1900, title_text=f"Promedios de temperatura en {city} (2020-2025)")

    fig_subplots.show() # Mostrar el gráfico

    # Mostrar promedios en consola
    print(f"Promedio de temperatura máxima (2020–2025): {promedio_total_calor:.2f} °C")
    print(f"Promedio de temperatura mínima (2020–2025): {promedio_total_frio:.2f} °C")
    print(f"Promedio de temperatura media (2020–2025): {promedio_total_medio:.2f} °C")
    print(f"Promedio de temperatura aparente (2020–2025): {promedio_total_aparente:.2f} °C")
    print(f"Promedio de amplitud térmica (2020–2025): {promedio_total_amplitud:.2f} °C")



##¿Cual es el periodo de tiempo (meses) que tienen mayor y menor temperatura de las ciudades con mayor temperatura?

In [ ]:
for city in ciudades:
    print(f"\n===== {city} =====")

    # Filtrar datos para la ciudad
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city]

    for año in df_ciudad['año'].unique():
        df_año = df_ciudad[df_ciudad['año'] == año]

        # Agrupar por mes y calcular promedios
        promedio_min = df_año.groupby('mes')['temperature_2m_min'].mean()
        promedio_max = df_año.groupby('mes')['temperature_2m_max'].mean()

        # Crear gráfico de barras combinado
        x = promedio_min.index
        width = 0.35  # ancho de las barras

        plt.figure(figsize=(12, 6))
        plt.bar(x - 0.175, promedio_min.values, width=width, color='orange', label='Temp. Mínima')
        plt.bar(x + 0.175, promedio_max.values, width=width, color='red', alpha=0.7, label='Temp. Máxima')

        plt.title(f'Temperaturas Promedio por Mes - {año} - {city}', fontsize=14)
        plt.xlabel('Mes', fontsize=12)
        plt.ylabel('Temperatura (°C)', fontsize=12)
        plt.legend()
        plt.xticks(ticks=x, labels=[f'Mes {m}' for m in x], rotation=45)
        plt.tight_layout()
        plt.show()

        # Detectar los 3 meses más cálidos y más fríos
        meses_maximos = promedio_max.sort_values(ascending=False).head(3)
        meses_minimos = promedio_min.sort_values(ascending=True).head(3)

        print(f"\nTop 3 meses más calurosos (Temp. máxima promedio más alta):")
        for mes, temp in meses_maximos.items():
            print(f"  Mes {mes}: {temp:.2f} °C")

        print(f"\nTop 3 meses más fríos (Temp. mínima promedio más baja):")
        for mes, temp in meses_minimos.items():
            print(f"  Mes {mes}: {temp:.2f} °C")


##¿Qué meses tienen más días con temperaturas agradables (entre 20 y 30 grados) en las ciudades más calurosas?

Un equipo internacional de investigadores de diversas universidades entre las que se encuentran el Instituto Tecnológico de Massachusetts (MIT por sus siglas en inglés); la Universidad de San Diego, en California; o la Universidad de de British Columbia, en Canadá, se ha atrevido a determinar cuál es la temperatura ideal, aquella con la que nos sentimos más felices. El equipo pudo comprobar que hay más expresiones positivas durante el día si los termómetros no bajan de los 20 grados centígrados pero tampoco suben de los 30. Cuanto más se alejan de estas franjas, las palabras son más negativas, sostienen. Una medida perfecta, señalan, serían los 25 grados.

###Periodos calidos

In [ ]:
nombres_meses = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo',
    4: 'Abril', 5: 'Mayo', 6: 'Junio',
    7: 'Julio', 8: 'Agosto', 9: 'Septiembre',
    10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}

for city in ciudades:
    print(f"\n===== {city} =====")
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city]

    for año in df_ciudad['año'].unique():
        df_año = df_ciudad[df_ciudad['año'] == año]

        # Calcular los 3 meses más calurosos
        promedio_max_por_mes = df_año.groupby('mes')['temperature_2m_max'].mean()
        meses_mas_calurosos = promedio_max_por_mes.sort_values(ascending=False).head(3).index.tolist()

        # Filtrar días de esos meses
        df_dias = df_año[df_año['mes'].isin(meses_mas_calurosos)][['dia', 'mes', 'temperature_2m_max', 'diferencia_aparente', 'amplitud_termica']].copy()

        # Clasificación climática
        def clasificar_clima(temp):
            if temp < 20:
                return 'Frío'
            elif 20 <= temp < 30:
                return 'Agradable'
            else:
                return 'Caluroso'

        df_dias['Clima'] = df_dias['temperature_2m_max'].apply(clasificar_clima)
        df_dias['Mes_nombre'] = df_dias['mes'].map(nombres_meses)

        # Crear tabla cruzada
        df_heat = pd.crosstab(df_dias['Clima'], df_dias['Mes_nombre'])

        # Asegurar orden de filas
        orden_clima = ['Frío', 'Agradable', 'Caluroso']
        df_heat = df_heat.reindex(index=orden_clima)

        # Crear heatmap
        plt.figure(figsize=(8, 6))
        sns.heatmap(df_heat, annot=True, cmap='viridis', fmt='.0f', cbar_kws={"label": "Cantidad de Días"})
        plt.title(f"Distribución de Climas - {city} ({año})")
        plt.xlabel('Mes')
        plt.ylabel('Tipo de Clima')
        plt.tight_layout()
        plt.show()


###Periodos frios

In [ ]:
nombres_meses = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo',
    4: 'Abril', 5: 'Mayo', 6: 'Junio',
    7: 'Julio', 8: 'Agosto', 9: 'Septiembre',
    10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}

for city in ciudades:
    print(f"\n===== {city} =====")
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city]

    for año in df_ciudad['año'].unique():
        df_año = df_ciudad[df_ciudad['año'] == año]

        # Calcular los 3 meses más calurosos
        promedio_min_por_mes = df_año.groupby('mes')['temperature_2m_min'].mean()
        meses_menos_calurosos = promedio_max_por_mes.sort_values(ascending=False).head(3).index.tolist()

        # Filtrar días de esos meses
        df_dias_frios = df_año[df_año['mes'].isin(meses_menos_calurosos)][['dia', 'mes', 'temperature_2m_min']].copy()

        # Clasificación climática
        def clasificar_clima(temp):
            if temp < 20:
                return 'Frío'
            elif 20 <= temp < 30:
                return 'Agradable'
            else:
                return 'Caluroso'

        df_dias_frios['Clima'] = df_dias_frios['temperature_2m_min'].apply(clasificar_clima)
        df_dias_frios['Mes_nombre'] = df_dias_frios['mes'].map(nombres_meses)

        # Crear tabla cruzada
        df_frios = pd.crosstab(df_dias_frios['Clima'], df_dias_frios['Mes_nombre']) # Change df_frio to df_frios

        # Asegurar orden de filas
        orden_clima = ['Frío', 'Agradable', 'Caluroso']
        df_frios = df_frios.reindex(index=orden_clima)

        # Crear heatmap
        plt.figure(figsize=(8, 6))
        sns.heatmap(df_frios, annot=True, cmap='viridis', fmt='.0f', cbar_kws={"label": "Cantidad de Días"})
        plt.title(f"Distribución de Climas - {city} ({año})")
        plt.xlabel('Mes')
        plt.ylabel('Tipo de Clima')
        plt.tight_layout()
        plt.show()

##¿Cómo se distribuye la diferencia entre la temperatura media y la sensación térmica (diferencia aparente) en estas ciudades?



In [ ]:
for city in ciudades:
    print(f"\n===== {city} =====")
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city].copy() # Use .copy() to avoid SettingWithCopyWarning

    # Ensure 'año', 'mes', and 'dia' columns exist in this specific df_ciudad
    # Re-extracting them from the 'datetime' column for robustness
    df_ciudad['año'] = df_ciudad['datetime'].dt.year
    df_ciudad['mes'] = df_ciudad['datetime'].dt.month
    df_ciudad['dia'] = df_ciudad['datetime'].dt.day

    # Rename columns to 'year', 'month', 'day' for to_datetime
    df_ciudad_renamed = df_ciudad.rename(columns={'año': 'year', 'mes': 'month', 'dia': 'day'})

    # Now, construct the 'fecha' column using the renamed columns
    # Pass the renamed DataFrame/columns to to_datetime
    df_ciudad['fecha'] = pd.to_datetime(df_ciudad_renamed[['year', 'month', 'day']])

    for año in df_ciudad['año'].unique():
        df_año = df_ciudad[df_ciudad['año'] == año].copy() # Use .copy() here too

        # Obtener número de semana ISO
        df_año['semana'] = df_año['fecha'].dt.isocalendar().week

        # Agrupar por semana y sacar media de diferencia_aparente
        df_semana = df_año.groupby('semana')['diferencia_aparente'].mean().reset_index()

        # Histograma o gráfico de barras para mostrar la diferencia media por semana
        fig = px.bar(df_semana, x='semana', y='diferencia_aparente',
                     title=f"Diferencia aparente media por semana en {city} - {año}",
                     labels={'semana': 'Semana del año', 'diferencia_aparente': 'Diferencia aparente'})
        fig.show()

##¿Cómo se distribuye la diferencia entre la temperatura máxima y mínima (amplitud térmica) en estas ciudades?

In [ ]:
for city in ciudades:
    print(f"\n===== {city} =====")
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city].copy() # Use .copy() to avoid SettingWithCopyWarning

    # Ensure 'año', 'mes', and 'dia' columns exist in this specific df_ciudad
    # Re-extracting them from the 'datetime' column for robustness
    df_ciudad['año'] = df_ciudad['datetime'].dt.year
    df_ciudad['mes'] = df_ciudad['datetime'].dt.month
    df_ciudad['dia'] = df_ciudad['datetime'].dt.day

    # Rename columns to 'year', 'month', 'day' for to_datetime
    df_ciudad_renamed = df_ciudad.rename(columns={'año': 'year', 'mes': 'month', 'dia': 'day'})

    # Now, construct the 'fecha' column using the renamed columns
    # Pass the renamed DataFrame/columns to to_datetime
    df_ciudad['fecha'] = pd.to_datetime(df_ciudad_renamed[['year', 'month', 'day']])

    for año in df_ciudad['año'].unique():
        df_año = df_ciudad[df_ciudad['año'] == año].copy() # Use .copy() here too

        # Obtener número de semana ISO
        df_año['semana'] = df_año['fecha'].dt.isocalendar().week

        # Agrupar por semana y sacar media de diferencia_aparente
        df_semana = df_año.groupby('semana')['amplitud_termica'].mean().reset_index()

        # Histograma o gráfico de barras para mostrar la diferencia media por semana
        fig = px.bar(df_semana, x='semana', y='amplitud_termica',
                     title=f"Diferencia aparente media por semana en {city} - {año}",
                     labels={'semana': 'Semana del año', 'amplitud_termica': 'amplitud_termica'})
        fig.show()

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
for city in ciudades:
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city].copy()
    df_filtrado = df_filtrado.dropna()


In [ ]:
for city in ciudades:
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city].copy()
    x = df_ciudad[['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean']]

In [ ]:
for city in ciudades:
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city].copy()
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)

In [ ]:
for city in ciudades:
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city].copy()
    kmeans = KMeans(n_clusters=3, random_state=42)
    kmeans.fit(x_scaled)
    df_ciudad['cluster'] = kmeans.labels_

In [ ]:
for city in ciudades:
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city].copy()
    # Add clustering and label assignment here
    x = df_ciudad[['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean']]
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10) # Added n_init for KMeans
    kmeans.fit(x_scaled)
    df_ciudad['cluster'] = kmeans.labels_

    centroids = kmeans.cluster_centers_
    centroids_original = scaler.inverse_transform(centroids)
    plt.figure(figsize=(8, 6))
    colors = ['red', 'green', 'blue']
    for i, color in zip(range(3), colors):
      cluster_data = df_ciudad[df_ciudad['cluster'] == i]
      # Assuming you want to plot the temperature data that was used for clustering
      plt.scatter(cluster_data['temperature_2m_mean'], cluster_data['temperature_2m_max'], c=color, label=f'Cluster {i}') # Modified to use temperature columns

    # Plotting centroids on the same axes as scatter plot
    plt.scatter(centroids_original[:, 2], centroids_original[:, 0], color='black', marker='X', s=200, label='Centroids', zorder=10) # Modified to use temperature columns

    plt.title(f'Clustering de k-means para {city} (con centroides)') # Added city name to title
    plt.xlabel('Temperatura Media (°C)') # Modified x-axis label
    plt.ylabel('Temperatura Máxima (°C)') # Modified y-axis label
    plt.legend()
    plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score

# Define y outside the loop using the 'clima_categoria' column
y = df_filtrado['clima_categoria']

for city in ciudades:
    df_ciudad = df_filtrado[df_filtrado['city_name'] == city].copy()
    # Define x for the current city within the loop
    x = df_ciudad[['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean']]
    # Filter y to match the current city's data
    y_ciudad = y[df_filtrado['city_name'] == city]

    # Ensure that x and y_ciudad have the same number of samples before splitting
    if len(x) == len(y_ciudad):
        x_train, x_test, y_train, y_test = train_test_split(x, y_ciudad, test_size=0.2, random_state=42)

        base_model = RandomForestClassifier(random_state=42)

        bagging_model = BaggingClassifier(estimator=base_model, n_estimators=10, random_state=42)
        bagging_model.fit(x_train, y_train)
        y_pred = bagging_model.predict(x_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Exactitud del modelo con random Forest para {city}: {accuracy:.4f}")
    else:
        print(f"Skipping {city} due to inconsistent sample sizes between features (x) and target (y).")

#Ciudades con menor promedio de temperatura

##¿Cómo varían las diferentes variables de temperatura (máxima, mínima, media, aparente) a lo largo del año en las ciudades más frias?

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Filtrar por año
df_filtrado_frio = df_filtrado_frio[df_filtrado_frio['año'].between(2020, 2025)]

# Obtener ciudades del dataframe filtrado
ciudades_frio = df_filtrado_frio['city_name'].unique()

# Iterar sobre cada ciudad
for city in ciudades_frio:
    print(f"\n===== {city} =====")

    # Filtrar datos para la ciudad
    df_ciudad_frio = df_filtrado_frio[df_filtrado_frio['city_name'] == city]

    # Agrupar por año y calcular promedios
    promedio_anual_calor = df_ciudad_frio.groupby('año')['temperature_2m_max'].mean()
    promedio_total_calor = promedio_anual_calor.mean()

    promedio_anual_frio = df_ciudad_frio.groupby('año')['temperature_2m_min'].mean()
    promedio_total_frio = promedio_anual_frio.mean()

    promedio_anual_medio = df_ciudad_frio.groupby('año')['temperature_2m_mean'].mean()
    promedio_total_medio = promedio_anual_medio.mean()

    promedio_anual_aparente = df_ciudad_frio.groupby('año')['apparent_temperature_mean'].mean()
    promedio_total_aparente = promedio_anual_aparente.mean()

    promedio_anual_amplitud = df_ciudad_frio.groupby('año')['amplitud_termica'].mean()
    promedio_total_amplitud = promedio_anual_amplitud.mean()

    # Crear gráfico de barras
    fig_subplots = make_subplots(
        rows=2, cols=3,
        subplot_titles=[
            "Promedio de calor por año",
            "Promedio de frío por año",
            "Promedio de temperatura media por año",
            "Temperatura aparente por año",
            "Amplitud térmica por año"
        ]
    )

    fig_subplots.add_trace(go.Bar(x=promedio_anual_calor.index, y=promedio_anual_calor, name="Temp. Máxima"), row=1, col=1)
    fig_subplots.add_trace(go.Bar(x=promedio_anual_frio.index, y=promedio_anual_frio, name="Temp. Mínima"), row=1, col=2)
    fig_subplots.add_trace(go.Bar(x=promedio_anual_medio.index, y=promedio_anual_medio, name="Temp. Media"), row=1, col=3)
    fig_subplots.add_trace(go.Bar(x=promedio_anual_aparente.index, y=promedio_anual_aparente, name="Temp. Aparente"), row=2, col=1)
    fig_subplots.add_trace(go.Bar(x=promedio_anual_amplitud.index, y=promedio_anual_amplitud, name="Amplitud Térmica"), row=2, col=2)

    fig_subplots.update_layout(height=700, width=1900, title_text=f"Promedios de temperatura en {city} (2020-2025)")
    fig_subplots.show()

    # Mostrar promedios en consola
    print(f"Promedio de temperatura máxima (2020–2025): {promedio_total_calor:.2f} °C")
    print(f"Promedio de temperatura mínima (2020–2025): {promedio_total_frio:.2f} °C")
    print(f"Promedio de temperatura media (2020–2025): {promedio_total_medio:.2f} °C")
    print(f"Promedio de temperatura aparente (2020–2025): {promedio_total_aparente:.2f} °C")
    print(f"Promedio de amplitud térmica (2020–2025): {promedio_total_amplitud:.2f} °C")

##¿Cuáles son los meses con mayor y menor temperatura en estas ciudades?

In [ ]:
for city in ciudades_frio:
    print(f"\n===== {city} =====")

    # Filtrar datos para la ciudad
    df_ciudad_frio = df_filtrado_frio[df_filtrado_frio['city_name'] == city]

    for año in df_ciudad_fria['año'].unique():
        df_año_fria = df_ciudad_fria[df_ciudad_fria['año'] == año]

        # Agrupar por mes y calcular promedios
        promedio_min_fria = df_año_fria.groupby('mes')['temperature_2m_min'].mean()
        promedio_max_fria = df_año_fria.groupby('mes')['temperature_2m_max'].mean()

        # Obtener el índice (meses) del promedio mínimo para la ciudad fría actual
        x = promedio_min_fria.index  # Changed this line to get the index from the current city's data
        width = 0.35  # ancho de las barras

        plt.figure(figsize=(12, 6))
        plt.bar(x - 0.175, promedio_min_fria.values, width=width, color='orange', label='Temp. Mínima')
        plt.bar(x + 0.175, promedio_max_fria.values, width=width, color='red', alpha=0.7, label='Temp. Máxima')

        plt.title(f'Temperaturas Promedio por Mes - {año} - {city}', fontsize=14)  # Changed this line to display the year correctly
        plt.xlabel('Mes', fontsize=12)
        plt.ylabel('Temperatura (°C)', fontsize=12)
        plt.legend()
        plt.xticks(ticks=x, labels=[f'Mes {m}' for m in x], rotation=45)
        plt.tight_layout()
        plt.show()

        # Detectar los 3 meses más cálidos y más fríos
        meses_maximos_fria = promedio_max_fria.sort_values(ascending=False).head(3)
        meses_minimos_fria = promedio_min_fria.sort_values(ascending=True).head(3)

        print(f"\nTop 3 meses más calurosos (Temp. máxima promedio más alta):")
        for mes, temp in meses_maximos_fria.items():
            print(f"  Mes {mes}: {temp:.2f} °C")

        print(f"\nTop 3 meses más fríos (Temp. mínima promedio más baja):")
        for mes, temp in meses_minimos_fria.items():
            print(f"  Mes {mes}: {temp:.2f} °C")

##¿Qué meses tienen más días con temperaturas agradables (entre 20 y 30 grados) en las ciudades más calurosas?

###Periodos calidos

In [ ]:
nombres_meses = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo',
    4: 'Abril', 5: 'Mayo', 6: 'Junio',
    7: 'Julio', 8: 'Agosto', 9: 'Septiembre',
    10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}

for city in ciudades_frio:
    print(f"\n===== {city} =====")
    df_ciudad_fria = df_filtrado_frio[df_filtrado_frio['city_name'] == city]

    for año in df_ciudad_fria['año'].unique():
        df_año = df_ciudad_fria[df_ciudad_fria['año'] == año]

        # Calcular los 3 meses más calurosos
        promedio_max_por_mes = df_año.groupby('mes')['temperature_2m_max'].mean()
        meses_mas_calurosos = promedio_max_por_mes.sort_values(ascending=False).head(3).index.tolist()

        # Filtrar días de esos meses
        df_dias = df_año[df_año['mes'].isin(meses_mas_calurosos)][['dia', 'mes', 'temperature_2m_max']].copy()

        # Clasificación climática
        def clasificar_clima(temp):
            if temp < 20:
                return 'Frío'
            elif 20 <= temp < 30:
                return 'Agradable'
            else:
                return 'Caluroso'

        df_dias['Clima'] = df_dias['temperature_2m_max'].apply(clasificar_clima)
        df_dias['Mes_nombre'] = df_dias['mes'].map(nombres_meses)

        # Crear tabla cruzada
        df_heat = pd.crosstab(df_dias['Clima'], df_dias['Mes_nombre'])

        # Asegurar orden de filas
        orden_clima = ['Frío', 'Agradable', 'Caluroso']
        df_heat = df_heat.reindex(index=orden_clima)

        # Crear heatmap
        plt.figure(figsize=(7, 5))
        sns.heatmap(df_heat, annot=True, cmap='viridis', fmt='.0f', cbar_kws={"label": "Cantidad de Días"})
        plt.title(f"Distribución de Climas - {city} ({año})")
        plt.xlabel('Mes')
        plt.ylabel('Tipo de Clima')
        plt.tight_layout()
        plt.show()

###Periodos frios

In [ ]:
nombres_meses = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo',
    4: 'Abril', 5: 'Mayo', 6: 'Junio',
    7: 'Julio', 8: 'Agosto', 9: 'Septiembre',
    10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}

for city in ciudades_frio:
    print(f"\n===== {city} =====")
    df_ciudad_fria = df_filtrado_frio[df_filtrado_frio['city_name'] == city]

    for año in df_ciudad_fria['año'].unique():
        df_año = df_ciudad_fria[df_ciudad_fria['año'] == año]

        # Calcular los 3 meses más calurosos
        promedio_min_por_mes = df_año.groupby('mes')['temperature_2m_min'].mean()
        meses_menos_calurosos = promedio_max_por_mes.sort_values(ascending=False).head(3).index.tolist()

        # Filtrar días de esos meses
        df_dias_frios = df_año[df_año['mes'].isin(meses_menos_calurosos)][['dia', 'mes', 'temperature_2m_min']].copy()

        # Clasificación climática
        def clasificar_clima(temp):
            if temp < 20:
                return 'Frío'
            elif 20 <= temp < 30:
                return 'Agradable'
            else:
                return 'Caluroso'

        df_dias_frios['Clima'] = df_dias_frios['temperature_2m_min'].apply(clasificar_clima)
        df_dias_frios['Mes_nombre'] = df_dias_frios['mes'].map(nombres_meses)

        # Crear tabla cruzada
        df_frios = pd.crosstab(df_dias_frios['Clima'], df_dias_frios['Mes_nombre']) # Change df_frio to df_frios

        # Asegurar orden de filas
        orden_clima = ['Frío', 'Agradable', 'Caluroso']
        df_frios = df_frios.reindex(index=orden_clima)

        # Crear heatmap
        plt.figure(figsize=(8, 6))
        sns.heatmap(df_frios, annot=True, cmap='viridis', fmt='.0f', cbar_kws={"label": "Cantidad de Días"})
        plt.title(f"Distribución de Climas - {city} ({año})")
        plt.xlabel('Mes')
        plt.ylabel('Tipo de Clima')
        plt.tight_layout()
        plt.show()


##¿Cómo se distribuye la diferencia entre la temperatura media y la sensación térmica (diferencia aparente) en estas ciudades?

In [ ]:
for city in ciudades_frio:
    print(f"\n===== {city} =====")
    df_ciudad_frio = df_filtrado_frio[df_filtrado_frio['city_name'] == city].copy()

    df_ciudad_frio['año'] = df_ciudad_frio['datetime'].dt.year
    df_ciudad_frio['mes'] = df_ciudad_frio['datetime'].dt.month
    df_ciudad_frio['dia'] = df_ciudad_frio['datetime'].dt.day

    df_ciudad_renamed_frio = df_ciudad_frio.rename(columns={'año': 'year', 'mes': 'month', 'dia': 'day'})

    df_ciudad_frio['fecha'] = pd.to_datetime(df_ciudad_renamed_frio[['year', 'month', 'day']])

    for año in df_ciudad_frio['año'].unique():
        df_año_frio = df_ciudad_frio[df_ciudad_frio['año'] == año].copy() # Use .copy() here too

        df_año_frio['semana'] = df_año_frio['fecha'].dt.isocalendar().week

        df_semana_frio = df_año_frio.groupby('semana')['diferencia_aparente'].mean().reset_index()

        fig = px.bar(df_semana_frio, x='semana', y='diferencia_aparente',
                     title=f"Diferencia aparente media por semana en {city} - {año}",
                     labels={'semana': 'Semana del año', 'diferencia_aparente': 'Diferencia aparente'})
        fig.show()

##¿Cómo se distribuye la diferencia entre la temperatura máxima y mínima (amplitud térmica) en estas ciudades?

In [ ]:
for city in ciudades_frio:
    print(f"\n===== {city} =====")
    df_ciudad_frio = df_filtrado_frio[df_filtrado_frio['city_name'] == city].copy()

    df_ciudad_frio['año'] = df_ciudad_frio['datetime'].dt.year
    df_ciudad_frio['mes'] = df_ciudad_frio['datetime'].dt.month
    df_ciudad_frio['dia'] = df_ciudad_frio['datetime'].dt.day

    df_ciudad_renamed_frio = df_ciudad_frio.rename(columns={'año': 'year', 'mes': 'month', 'dia': 'day'})

    df_ciudad_frio['fecha'] = pd.to_datetime(df_ciudad_renamed_frio[['year', 'month', 'day']])

    for año in df_ciudad_frio['año'].unique():
        df_año_frio = df_ciudad_frio[df_ciudad_frio['año'] == año].copy()

        df_año_frio['semana'] = df_año_frio['fecha'].dt.isocalendar().week

        df_semana_frio = df_año_frio.groupby('semana')['amplitud_termica'].mean().reset_index()

        fig = px.bar(df_semana_frio, x='semana', y='amplitud_termica',
                     title=f"Diferencia aparente media por semana en {city} - {año}",
                     labels={'semana': 'Semana del año', 'amplitud_termica': 'Amplitud termica'})
        fig.show()

In [ ]:
for city in ciudades_frio:
    df_ciudad_fria = df_filtrado_frio[df_filtrado_frio['city_name'] == city]
    df_filtrado_frio = df_filtrado_frio.dropna()


In [ ]:
for city in ciudades_frio:
    df_ciudad_fria = df_filtrado_frio[df_filtrado_frio['city_name'] == city]
    x = df_ciudad_fria[['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean']]

In [ ]:
for city in ciudades_frio:
    df_ciudad_fria = df_filtrado_frio[df_filtrado_frio['city_name'] == city]
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)

In [ ]:
for city in ciudades_frio:
    df_ciudad_fria = df_filtrado_frio[df_filtrado_frio['city_name'] == city]
    kmeans = KMeans(n_clusters=3, random_state=42)
    kmeans.fit(x_scaled)
    df_ciudad_fria['cluster'] = kmeans.labels_

In [ ]:
for city in ciudades_frio:
    df_ciudad_fria = df_filtrado_frio[df_filtrado_frio['city_name'] == city]
    # Add clustering and label assignment here
    x = df_ciudad_fria[['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean']]
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10) # Added n_init for KMeans
    kmeans.fit(x_scaled)
    df_ciudad_fria['cluster'] = kmeans.labels_

    centroids = kmeans.cluster_centers_
    centroids_original = scaler.inverse_transform(centroids)
    plt.figure(figsize=(8, 6))
    colors = ['red', 'green', 'blue']
    for i, color in zip(range(3), colors):
      cluster_data = df_ciudad_fria[df_ciudad_fria['cluster'] == i]
      # Assuming you want to plot the temperature data that was used for clustering
      plt.scatter(cluster_data['temperature_2m_mean'], cluster_data['temperature_2m_max'], c=color, label=f'Cluster {i}') # Modified to use temperature columns

    # Plotting centroids on the same axes as scatter plot
    plt.scatter(centroids_original[:, 2], centroids_original[:, 0], color='black', marker='X', s=200, label='Centroids', zorder=10) # Modified to use temperature columns

    plt.title(f'Clustering de k-means para {city} (con centroides)') # Added city name to title
    plt.xlabel('Temperatura Media (°C)') # Modified x-axis label
    plt.ylabel('Temperatura Máxima (°C)') # Modified y-axis label
    plt.legend()
    plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score

# Define y outside the loop using the 'clima_categoria' column
y = df_filtrado_frio['clima_categoria']

for city in ciudades_frio:
    df_ciudad_fria = df_filtrado_frio[df_filtrado_frio['city_name'] == city]
    # Define x for the current city within the loop
    x = df_ciudad_fria[['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean']]
    # Filter y to match the current city's data
    y_ciudad = y[df_filtrado_frio['city_name'] == city]

    # Ensure that x and y_ciudad have the same number of samples before splitting
    if len(x) == len(y_ciudad):
        x_train, x_test, y_train, y_test = train_test_split(x, y_ciudad, test_size=0.2, random_state=42)

        base_model = RandomForestClassifier(random_state=42)

        bagging_model = BaggingClassifier(estimator=base_model, n_estimators=10, random_state=42)
        bagging_model.fit(x_train, y_train)
        y_pred = bagging_model.predict(x_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Exactitud del modelo con random Forest para {city}: {accuracy:.4f}")
    else:
        print(f"Skipping {city} due to inconsistent sample sizes between features (x) and target (y).")

#Agricultura

##¿Cuáles son las ciudades con las mejores condiciones agroclimáticas en general, considerando temperatura, precipitación, radiación solar y balance hídrico?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df1['datetime'] = pd.to_datetime(df1['datetime'])

promedio_por_ciudad_calor = df1.groupby('city_name')['temperature_2m_mean'].mean()

top_promedios = promedio_por_ciudad_calor[
    (promedio_por_ciudad_calor > 17) & (promedio_por_ciudad_calor < 27)
]

top_promedios = top_promedios.sort_values(ascending=False)

print("Ciudades con promedio de temperatura máxima entre 17°C y 27°C:")

top_ciudades_agro = top_promedios.reset_index()


plt.figure(figsize=(20, 12))
plt.barh(top_ciudades_agro['city_name'], top_ciudades_agro['temperature_2m_mean'], color='green')
plt.xlabel('Promedio de clima')
plt.title('ciudades con clima aptas para la agricultura')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
import pandas as pd

df1['datetime'] = pd.to_datetime(df1['datetime'])

df_humedad = df2[['city_name', 'datetime', 'relative_humidity_2m', 'dew_point_2m']].copy()
df_humedad['datetime'] = pd.to_datetime(df_humedad['datetime'])

df_humedad['date'] = pd.to_datetime(df_humedad['datetime'].dt.date)

humedad_diaria = df_humedad.groupby(['date', 'city_name'])['relative_humidity_2m'].mean().reset_index()
humedad_diaria.rename(columns={'relative_humidity_2m': 'humedad_relativa_promedio'}, inplace=True)

temp_diaria = df1.groupby(['datetime', 'city_name'])[['temperature_2m_mean',
                                                      'shortwave_radiation_sum',
                                                      'wind_speed_10m_max',
                                                      'wind_gusts_10m_max',
                                                      'wind_direction_10m_dominant',
                                                      'precipitation_sum',
                                                      'rain_sum',
                                                      'snowfall_sum',
                                                      'precipitation_hours',
                                                      'año',
                                                      'mes',
                                                      'dia',
                                                      'semana',
                                                      ]].mean().reset_index()
# Rename the 'datetime' column to 'date' for merging
temp_diaria.rename(columns={'datetime': 'date'}, inplace=True)

# Merge the daily temperature/radiation/wind data with daily humidity data
df_agricultura = pd.merge(humedad_diaria, temp_diaria, on=['date', 'city_name'], how='inner')

ciudades = top_promedios.index.unique()



for city in ciudades:
    df_ciudades_agricultura = df_agricultura[df_agricultura['city_name'] == city].copy()

    df_pivot_agricultura = df_ciudades_agricultura.set_index('date').sort_index()[[
        'año',
        'mes',
        'dia',
        'semana',
        'humedad_relativa_promedio',
        'temperature_2m_mean',
        'shortwave_radiation_sum',
        'wind_speed_10m_max',
        'wind_gusts_10m_max',
        'wind_direction_10m_dominant',
        'precipitation_sum',
        'rain_sum',
        'snowfall_sum',
        'precipitation_hours'
    ]]



##¿Cuáles son las ciudades con las mejores condiciones agroclimáticas en general, considerando temperatura, precipitación, radiación solar y balance hídrico?

Queremos ver cómo varían la temperatura media y la precipitación a lo largo del tiempo (mes a mes o semana a semana) para cada ciudad. Este análisis puede ayudar a determinar las temporadas más cálidas o húmedas para las actividades agrícolas.

###Humedad


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import math

from sklearn.model_selection import train_test_split

ciudades_agro = top_promedios.index  # Lista de ciudades a analizar

for city in ciudades_agro:
    print(f"\n===== {city} =====")

    df_ciudades_agricultura = df_agricultura[df_agricultura['city_name'] == city].copy()
    unique_years = df_ciudades_agricultura['año'].unique()
    num_years = len(unique_years)

    if num_years > 0:
        n_cols = min(num_years, 3)
        n_rows = math.ceil(num_years / n_cols)

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 6, n_rows * 5))
        axes = axes.flatten()

        for i, año in enumerate(unique_years):
            df_año_agro = df_ciudades_agricultura[df_ciudades_agricultura['año'] == año].copy()

            if 'humedad_relativa_promedio' in df_año_agro.columns:
                # Crear condición para análisis (opcional, usada como variable categórica)
                df_año_agro['condicion_color'] = df_año_agro['humedad_relativa_promedio'].between(20, 60)

            # Graficar si las columnas necesarias existen
            if 'temperature_2m_mean' in df_año_agro.columns and 'mes' in df_año_agro.columns:
                sns.scatterplot(
                    data=df_año_agro,
                    x='humedad_relativa_promedio',
                    y='temperature_2m_mean',
                    hue='mes',
                    palette='Spectral',
                    ax=axes[i],
                )
                axes[i].set_title(f"Año {año}")
                axes[i].set_xlabel('Humedad Relativa Promedio')
                axes[i].set_ylabel('Temperatura Media (°C)')
            else:
                print(f"No se puede graficar para {city} ({año}): faltan columnas necesarias.")

        # Eliminar subgráficos vacíos si hay menos años que subplots
        for j in range(num_years, len(axes)):
            fig.delaxes(axes[j])

        fig.suptitle(f"Humedad vs Temperatura en {city} por Año", fontsize=16, y=1.02)
        plt.tight_layout()
        plt.show()

    else:
        print(f"No hay datos disponibles por año para {city}.")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


ciudades_agro = top_promedios.index

for city in ciudades_agro:
    print(f"\n===== {city} =====")

    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    df_city = df_city[['temperature_2m_mean', 'humedad_relativa_promedio']].dropna()

    if not df_city.empty:
        X = df_city[['temperature_2m_mean']]
        y = df_city['humedad_relativa_promedio']

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = LinearRegression()
        model.fit(x_train, y_train)

        y_pred = model.predict(x_test)
        r2 = model.score(x_test, y_test)

        print(f"R-squared for {city}: {r2:.2f}")

        plt.figure(figsize=(8, 6))
        plt.scatter(y_test, y_pred, color='green', label='Predicciones')
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', label='Ajuste perfecto')
        plt.xlabel('Valores reales de humedad')
        plt.ylabel('Valores predichos de humedad')
        plt.title(f'Modelo Lineal - {city}')
        plt.legend()
        plt.tight_layout()
        plt.show()
    else:
        print(f" No hay datos suficientes para {city} después de eliminar NaNs.")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier


# Definimos categorías de humedad
bins = [0, 40, 70, 100]
labels = ['Baja', 'Media', 'Alta']

print("Columnas disponibles:", df_agricultura.columns)

for city in ciudades_agro:
    print(f"\n===== {city} =====")

    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    if not df_city.empty:
        # Filtramos columnas necesarias
        df_city = df_city[['temperature_2m_mean', 'humedad_relativa_promedio']].dropna()

        # Convertimos a numérico por seguridad
        df_city['humedad_relativa_promedio'] = pd.to_numeric(
            df_city['humedad_relativa_promedio'], errors='coerce'
        )
        df_city = df_city.dropna()

        # Categorizamos la humedad
        df_city['humedad_categoria'] = pd.cut(
            df_city['humedad_relativa_promedio'], bins=bins, labels=labels, right=False
        )

        # Eliminamos filas sin categoría
        df_city = df_city.dropna(subset=['humedad_categoria'])

        # Verificamos que hay al menos 2 clases con suficientes muestras
        if df_city['humedad_categoria'].nunique() < 2 or df_city['humedad_categoria'].value_counts().min() < 2:
            print(" No hay suficientes clases o muestras por clase para entrenar.")
            continue

        # Definimos X e y
        X = df_city[['temperature_2m_mean']]
        y = df_city['humedad_categoria']

        # Dividimos los datos
        x_train, x_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        # Entrenamos el modelo
        classifier = RandomForestClassifier(random_state=42)
        classifier.fit(x_train, y_train)

        # Hacemos predicciones
        y_pred = classifier.predict(x_test)

        # Matriz de confusión
        cm = confusion_matrix(y_test, y_pred, labels=labels)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

        # Mostramos la gráfica
        fig, ax = plt.subplots(figsize=(6, 6))
        disp.plot(ax=ax, cmap=plt.cm.Blues, values_format='d')
        plt.title(f'Matriz de Confusión - {city}')
        plt.xlabel('Predicción')
        plt.ylabel('Realidad')
        plt.tight_layout()
        plt.show()
    else:
        print(f" No hay datos suficientes para {city}.")




###Velocidad del viento

In [ ]:
import plotly.express as px

ciudades_agro = top_promedios.index

for city in ciudades_agro:
    print(f"\n===== {city} =====")

    df_ciudades_agricultura_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    # Eliminar filas con valores faltantes en las columnas relevantes para el gráfico
    df_ciudades_agricultura_city = df_ciudades_agricultura_city.dropna(subset=['humedad_relativa_promedio', 'temperature_2m_mean', 'wind_speed_10m_max'])

    if not df_ciudades_agricultura_city.empty:
        fig = px.scatter(df_ciudades_agricultura_city,
                         x="humedad_relativa_promedio",
                         y="temperature_2m_mean",
                         size="wind_speed_10m_max",
                         color="año", # Usar el año para diferenciar los puntos de diferentes años
                         hover_name="date", # Mostrar la fecha al pasar el mouse
                         title=f"Humedad vs Temperatura (Tamaño = Velocidad del Viento) en {city} por Año",
                         labels={"humedad_relativa_promedio": "Humedad Relativa Promedio (%)",
                                 "temperature_2m_mean": "Temperatura Media (°C)",
                                 "wind_speed_10m_max": "Velocidad Máxima del Viento (m/s)"},
                         animation_frame="año", # Animar por año
                         range_x=[0, 100], # Rango típico para humedad relativa
                         # Considerar ajustar el rango y para temperatura según los datos de las ciudades calurosas
                         # range_y=[min_temp, max_temp]
                        )
        fig.update_layout(showlegend=True)
        fig.show()
    else:
        print(f" No hay datos suficientes para graficar la relación de viento, humedad y temperatura en {city} después de eliminar NaNs.")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score # Importar métricas adicionales
import numpy as np # Importar numpy para la raíz cuadrada

# Lista de ciudades que quieres analizar
ciudades_agro = top_promedios.index  # Asegúrate de que 'top_promedios' sea un DataFrame o Series con índices de tipo string (nombres de ciudad)

for city in ciudades_agro:
    print(f"\n===== {city} =====")

    # Filtrar por ciudad
    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    # Verificar que existan todas las columnas necesarias
    required_columns = ['wind_speed_10m_max', 'wind_gusts_10m_max', 'humedad_relativa_promedio']
    if not all(col in df_city.columns for col in required_columns):
        print(f"Faltan columnas necesarias en los datos de {city}")
        continue

    # Eliminar filas con valores faltantes en las columnas relevantes
    df_city = df_city[required_columns].dropna()

    # Continuar solo si hay suficientes datos
    if not df_city.empty and len(df_city) >= 10:  # puedes ajustar el mínimo de filas requeridas
        X = df_city[['wind_speed_10m_max', 'wind_gusts_10m_max']]
        y = df_city['humedad_relativa_promedio']

        # Dividir datos en entrenamiento y prueba
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Entrenar modelo de regresión lineal
        model = LinearRegression()
        model.fit(x_train, y_train)

        # Predecir y evaluar
        y_pred = model.predict(x_test)

        # Calcular métricas de evaluación
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse) # Calcular RMSE como la raíz cuadrada del MSE

        # Mostrar resultados
        print(f"R-squared para {city}: {r2:.2f}")
        print(f"Mean Squared Error (MSE) para {city}: {mse:.2f}")
        print(f"Root Mean Squared Error (RMSE) para {city}: {rmse:.2f}")


        # Gráfico de resultados reales vs. predichos
        plt.figure(figsize=(8, 6))
        plt.scatter(y_test, y_pred, color='green', label='Predicciones')
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
                 color='red', linestyle='--', label='Ajuste perfecto')
        plt.xlabel('Valores reales de humedad')
        plt.ylabel('Valores predichos de humedad')
        plt.title(f'Modelo Lineal - {city}')
        plt.legend()
        plt.tight_layout()
        plt.grid(True)
        plt.show()

    else:
        print(f"No hay suficientes datos para {city} después de eliminar NaNs.")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier


# Definimos categorías de velocidad del viento
bins_wind = [0, 10, 20, np.inf] # Define wind speed categories (adjust bins as needed)
labels_wind = ['Bajo', 'Medio', 'Alto']

print("Columnas disponibles:", df_agricultura.columns)

for city in ciudades_agro:
    print(f"\n===== {city} =====")

    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    if not df_city.empty:
        # Filtramos columnas necesarias
        df_city = df_city[['temperature_2m_mean', 'wind_speed_10m_max', 'wind_gusts_10m_max']].dropna()

        # Convertimos a numérico por seguridad
        df_city[['wind_speed_10m_max', 'wind_gusts_10m_max']] = df_city[['wind_speed_10m_max', 'wind_gusts_10m_max']].apply(pd.to_numeric, errors='coerce')
        df_city = df_city.dropna()

        # Categorizamos la velocidad del viento (usando wind_speed_10m_max como ejemplo)
        df_city['wind_category'] = pd.cut(
            df_city['wind_speed_10m_max'], bins=bins_wind, labels=labels_wind, right=False
        )

        # Eliminamos filas sin categoría
        df_city = df_city.dropna(subset=['wind_category'])

        # Verificamos que hay al menos 2 clases con suficientes muestras
        if df_city['wind_category'].nunique() < 2 or df_city['wind_category'].value_counts().min() < 2:
            print(" No hay suficientes clases o muestras por clase para entrenar.")
            continue

        # Definimos X e y
        # Using both wind speed and gusts as features
        X = df_city[['temperature_2m_mean', 'wind_gusts_10m_max']]
        y = df_city['wind_category']

        # Dividimos los datos
        x_train, x_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        # Entrenamos el modelo
        classifier = RandomForestClassifier(random_state=42)
        classifier.fit(x_train, y_train)

        # Hacemos predicciones
        y_pred = classifier.predict(x_test)

        # Matriz de confusión
        cm = confusion_matrix(y_test, y_pred, labels=labels_wind) # Use wind labels
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels_wind) # Use wind labels

        # Mostramos la gráfica
        fig, ax = plt.subplots(figsize=(6, 6))
        disp.plot(ax=ax, cmap=plt.cm.Blues, values_format='d')
        plt.title(f'Matriz de Confusión - {city}')
        plt.xlabel('Predicción')
        plt.ylabel('Realidad')
        plt.tight_layout()
        plt.show()
    else:
        print(f" No hay datos suficientes para {city}.")

###Radiacion solar

In [ ]:
for city in ciudades_agro:

    # Filtrar datos por ciudad
    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()


In [ ]:
for city in ciudades_agro:

    # Filtrar datos por ciudad
    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    df_city['estado_de_ciudad'] = df1['clima_categoria']

In [ ]:
import plotly.express as px

for city in ciudades_agro:
    print(f"\n===== {city} =====")

    # Filtrar datos por ciudad
    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    # Asegurarse de que la columna 'date' existe
    if 'date' not in df_city.columns:
        df_city['date'] = pd.to_datetime(df_city[['año', 'mes', 'dia']])

    # Filtrar columnas relevantes y eliminar filas con valores nulos
    df_radiacion_solar = df_city[['shortwave_radiation_sum', 'temperature_2m_mean', 'año', 'date']].dropna()

    if not df_radiacion_solar.empty:
        fig = px.scatter(
            df_radiacion_solar,
            x="shortwave_radiation_sum",
            y="temperature_2m_mean",
            color="año",  # Diferenciar por año
            hover_name="date",  # Mostrar la fecha
            animation_frame="año",  # Crear animación por año
            title=f"Radiación Solar vs Temperatura en {city} por Año",
            labels={
                "shortwave_radiation_sum": "Radiación Solar (Wh/m² o MJ/m²)",
                "temperature_2m_mean": "Temperatura Media (°C)"
            },
            range_x=[0, 40],  # Ajusta según tus datos reales
            # range_y=[min_temp, max_temp]  # Opcional: puedes definir un rango si lo deseas
        )

        fig.update_layout(showlegend=True)
        fig.show()

    else:
        print(f"No hay datos suficientes para graficar la relación de radiación y temperatura en {city}.")


In [ ]:
for city in ciudades_agro:
    print(f"\n===== {city} =====")

    # Filtrar por ciudad
    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    # Verificar que existan todas las columnas necesarias
    required_columns = ['shortwave_radiation_sum', 'humedad_relativa_promedio']
    if not all(col in df_city.columns for col in required_columns):
        print(f"Faltan columnas necesarias en los datos de {city}")
        continue

    # Eliminar filas con valores faltantes en las columnas relevantes
    df_city = df_city[required_columns].dropna()

    # Continuar solo si hay suficientes datos
    if not df_city.empty and len(df_city) >= 10:  # puedes ajustar el mínimo de filas requeridas
        X = df_city[['shortwave_radiation_sum']]
        y = df_city['humedad_relativa_promedio']

        # Dividir datos en entrenamiento y prueba
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Entrenar modelo de regresión lineal
        model = LinearRegression()
        model.fit(x_train, y_train)

        # Predecir y evaluar
        y_pred = model.predict(x_test)

        # Calcular métricas de evaluación
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse) # Calcular RMSE como la raíz cuadrada del MSE

        # Mostrar resultados
        print(f"R-squared para {city}: {r2:.2f}")
        print(f"Mean Squared Error (MSE) para {city}: {mse:.2f}")
        print(f"Root Mean Squared Error (RMSE) para {city}: {rmse:.2f}")


        # Gráfico de resultados reales vs. predichos
        plt.figure(figsize=(8, 6))
        plt.scatter(y_test, y_pred, color='green', label='Predicciones')
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
                 color='red', linestyle='--', label='Ajuste perfecto')
        plt.xlabel('Valores reales de humedad')
        plt.ylabel('Valores predichos de humedad')
        plt.title(f'Modelo Lineal - {city}')
        plt.legend()
        plt.tight_layout()
        plt.grid(True)
        plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier

# Asegúrate de que ciudades_agro y df_agricultura estén definidos
# ciudades_agro = df_agricultura['city_name'].unique()

for city in ciudades_agro:
    print(f"\n===== {city} =====")

    # Filtrar datos por ciudad
    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    # Eliminar NaNs
    df_city = df_city[['shortwave_radiation_sum', 'temperature_2m_mean']].dropna()

    # Definir X y binarizar Y
    X = df_city[['shortwave_radiation_sum']]
    y = df_city['temperature_2m_mean'] > df_city['temperature_2m_mean'].median()
    y = y.astype(int)  # 1: temperatura alta, 0: baja

    # División de datos
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar clasificador
    classifier = RandomForestClassifier(random_state=42)
    classifier.fit(x_train, y_train)

    # Predicciones
    y_pred = classifier.predict(x_test)

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=('Baja', 'Alta'))

    fig, ax = plt.subplots(figsize=(8, 9))
    disp.plot(ax=ax, cmap=plt.cm.Blues)
    plt.title(f'Matriz de Confusión - {city}')
    plt.xlabel('Predicción')
    plt.ylabel('Realidad')

    # Añadir anotaciones personalizadas
    plt.text(0, 0, f'{cm[0,0]}\nTN\n(Verdaderos Negativos)', ha='center', va='center', fontsize=10, color='white')
    plt.text(1, 0, f'{cm[0,1]}\nFP\n(Falsos Positivos)', ha='center', va='center', fontsize=10, color='black')
    plt.text(0, 1, f'{cm[1,0]}\nFN\n(Falsos Negativos)', ha='center', va='center', fontsize=10, color='black')
    plt.text(1, 1, f'{cm[1,1]}\nTP\n(Verdaderos Positivos)', ha='center', va='center', fontsize=10, color='white')

    plt.tight_layout()
    plt.show()


###Lluvia

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

for city in ciudades_agro:
    print(f"\n===== {city} =====")

    # Filtrar datos por ciudad
    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    # Eliminar nulos y seleccionar columnas relevantes
    df_city_precipitacion = df_city[['date', 'año', 'mes', 'dia', 'precipitation_sum', 'temperature_2m_mean',
                                     'rain_sum', 'snowfall_sum', 'precipitation_hours']].dropna()

    # Asegurar que 'date' es datetime y usarla como índice
    df_city_precipitacion['date'] = pd.to_datetime(df_city_precipitacion['date'])
    df_city_precipitacion.set_index('date', inplace=True)

    # Agrupación trimestral (resample por trimestre)
    precipitacion_trimestral = df_city_precipitacion['precipitation_sum'].resample('Q').sum()

    # Graficar
    plt.figure(figsize=(15, 7))
    plt.plot(precipitacion_trimestral.index, precipitacion_trimestral.values, marker='o', linestyle='-')
    plt.title(f"Cantidad de lluvia por trimestre en {city}", fontsize=14)
    plt.xlabel("Fecha", fontsize=12)
    plt.ylabel("Precipitación acumulada (mm)", fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    plt.show()



In [ ]:
for city in ciudades_agro:
    print(f"\n===== {city} =====")


    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    df_city['date'] = pd.to_datetime(df_city['date'])

    df_city_precipitacion = df_city[['date', 'precipitation_sum']].dropna()

    df_city_precipitacion.set_index('date', inplace=True)

    df_city_precipitacion['lluvia_dia'] = df_city_precipitacion['precipitation_sum'] > 0

    dias_lluviosos = df_city_precipitacion.groupby(df_city_precipitacion.index.month)['lluvia_dia'].sum()

    print(dias_lluviosos)

    dias_lluviosos.plot(kind='bar', figsize=(10, 5), title=f'Días con lluvia por mes en {city}')
    plt.xlabel('Mes')
    plt.ylabel('Días con lluvia')
    plt.xticks(rotation=0)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

for city in ciudades_agro:
    print(f"\n===== {city} =====")

    # Filtrar datos por ciudad
    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    # Asegurarse de que 'date' sea datetime y poner como índice
    df_city['date'] = pd.to_datetime(df_city['date'])
    df_city.set_index('date', inplace=True)

    # Filtrar días con lluvia
    df_lluvia = df_city[df_city['precipitation_sum'] > 0].copy()

    # Función para clasificar intensidad de lluvia
    def clasificar_clima(valor):
        if valor < 5:
            return 'Lluvia ligera'
        elif 5 <= valor < 20:
            return 'Lluvia moderada'
        elif 20 <= valor < 50:
            return 'Lluvia fuerte'
        else:
            return 'Lluvia extrema'

    # Aplicar la clasificación a cada día con lluvia
    df_lluvia['clasificacion_lluvia'] = df_lluvia['precipitation_sum'].apply(clasificar_clima)

    # Mostrar resumen por año
    resumen = df_lluvia.groupby([df_lluvia.index.year, 'clasificacion_lluvia']).size().unstack(fill_value=0)

    # Opcional: gráfico de barras apiladas por año
    resumen.plot(kind='bar', stacked=True, figsize=(12, 6), title=f"Clasificación de lluvia por año en {city}")
    plt.xlabel("Año")
    plt.ylabel("Cantidad de días")
    plt.grid(True)
    plt.tight_layout()
    plt.show()








In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
for city in ciudades_agro:
    print(f"\n===== {city} =====")

    # Filtrar datos por ciudad
    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    # Asegurar que 'date' es datetime
    df_city['date'] = pd.to_datetime(df_city['date'])
    df_city.set_index('date', inplace=True)

    # Filtrar días con precipitación
    df_lluvia = df_city[df_city['precipitation_sum'] > 0].copy()

    # Clasificar tipo de lluvia
    def clasificar_clima(valor):
        if valor < 5:
            return 'Lluvia ligera'
        elif 5 <= valor < 20:
            return 'Lluvia moderada'
        elif 20 <= valor < 50:
            return 'Lluvia fuerte'
        else:
            return 'Lluvia extrema'

    df_lluvia['clasificacion_lluvia'] = df_lluvia['precipitation_sum'].apply(clasificar_clima)

    # Seleccionar solo variables numéricas para PCA
    columnas_numericas = [
        'temperature_2m_mean',
        'shortwave_radiation_sum',
        'wind_speed_10m_max',
        'wind_gusts_10m_max',
        'precipitation_sum',
        'rain_sum',
        'snowfall_sum',
        'precipitation_hours',
        'humidity_relativa_promedio'
    ]

    # Filtrar columnas presentes
    columnas_usar = [col for col in columnas_numericas if col in df_lluvia.columns]
    df_lluvia_clean = df_lluvia.dropna(subset=columnas_usar + ['clasificacion_lluvia'])

    x = df_lluvia_clean[columnas_usar]
    y = df_lluvia_clean['clasificacion_lluvia']

    # Estandarizar
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)

    # Aplicar PCA
    pca = PCA(n_components=2)
    x_pca = pca.fit_transform(x_scaled)

    print("Varianza explicada por los 2 primeros componentes:", pca.explained_variance_ratio_)

    # Crear DataFrame con componentes principales y clases
    df_pca = pd.DataFrame(data=x_pca, columns=['PC1', 'PC2'])
    df_pca['clasificacion_lluvia'] = y.values

    # Visualizar
    plt.figure(figsize=(8, 6))
    colores = {
        'Lluvia ligera': 'blue',
        'Lluvia moderada': 'green',
        'Lluvia fuerte': 'orange',
        'Lluvia extrema': 'red'
    }

    for clase, color in colores.items():
        subset = df_pca[df_pca['clasificacion_lluvia'] == clase]
        plt.scatter(subset['PC1'], subset['PC2'], label=clase, alpha=0.6, color=color)

    plt.title(f'PCA de días con lluvia en {city}')
    plt.xlabel('Componente Principal 1')
    plt.ylabel('Componente Principal 2')
    plt.legend(title="Clasificación de lluvia")
    plt.grid(True)
    plt.tight_layout()
    plt.show()



###Nevadas

In [ ]:
for city in ciudades_agro:
    print(f"\n===== {city} =====")

    # Filtrar datos por ciudad
    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    df_city_precipitacion = df_city[['date', 'año', 'mes', 'dia', 'precipitation_sum', 'temperature_2m_mean',
                                     'rain_sum', 'snowfall_sum', 'precipitation_hours']].dropna()

    # Asegurar que 'date' es datetime y usarla como índice
    df_city_precipitacion['date'] = pd.to_datetime(df_city_precipitacion['date'])
    df_city_precipitacion.set_index('date', inplace=True)

    # Agrupación trimestral (resample por trimestre)
    precipitacion_trimestral_nevada = df_city_precipitacion['snowfall_sum'].resample('Q').sum()

    # Graficar
    plt.figure(figsize=(15, 7))
    plt.plot(precipitacion_trimestral_nevada.index, precipitacion_trimestral_nevada.values, marker='o', linestyle='-')
    plt.title(f"Cantidad de lluvia por trimestre en {city}", fontsize=14)
    plt.xlabel("Fecha", fontsize=12)
    plt.ylabel("Precipitación acumulada (mm)", fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    plt.show()




In [ ]:
from matplotlib import pyplot as plt
import pandas as pd

# Función para clasificar intensidad de nevada
def clasificar_nevada(valor):
    if 0.1 <= valor < 2.5:
        return 'Nevada ligera'
    elif 2.5 <= valor < 7.5:
        return 'Nevada moderada'
    elif 7.5 <= valor < 15:
        return 'Nevada fuerte'
    else:
        return 'Nevada extrema'

# Iterar sobre cada ciudad
for city in ciudades_agro:
    print(f"\n===== {city} =====")

    # Filtrar datos por ciudad
    df_city = df_agricultura[df_agricultura['city_name'] == city].copy()

    # Asegurar que 'date' esté en formato datetime y usarlo como índice
    df_city['date'] = pd.to_datetime(df_city['date'])
    df_city.set_index('date', inplace=True)

    # Filtrar días con nieve
    df_nevada = df_city[df_city['snowfall_sum'] > 0].copy()

    if df_nevada.empty:
        print(f"No hay días con nieve registrados en {city}.")
        continue

    # Clasificar intensidad de la nevada
    df_nevada['clasificacion_nieve'] = df_nevada['snowfall_sum'].apply(clasificar_nevada)

    # Resumen por año y categoría
    resumen = df_nevada.groupby([df_nevada.index.year, 'clasificacion_nieve']).size().unstack(fill_value=0)

    # Gráfico de barras apiladas
    resumen.plot(kind='bar', stacked=True, figsize=(12, 6),
                 title=f"Clasificación de nevadas por año en {city}")
    plt.xlabel("Año")
    plt.ylabel("Cantidad de días con nevada")
    plt.grid(True)
    plt.tight_layout()
    plt.show()



##¿Que cosecha se puede cultivar en cada ciudad

In [ ]:
df4.groupby('Crop')[['Temperature_Celsius', 'Rainfall_mm']].agg(['mean', 'min', 'max'])

In [ ]:
df1['year'] = pd.to_datetime(df1['datetime']).dt.year
weather_summary = df1.groupby(['city_name', 'year'])[['temperature_2m_mean', 'precipitation_sum']].mean().reset_index()

In [ ]:
def is_suitable(temp, rain, crop):
    crop_data = crop_conditions[crop]
    return (
        crop_data['temp_min'] <= temp <= crop_data['temp_max'] and
        crop_data['rain_min'] <= rain <= crop_data['rain_max']
    )

In [ ]:
crop_conditions = df4.groupby('Crop')[['Temperature_Celsius', 'Rainfall_mm']].agg(['mean', 'min', 'max'])
crop_conditions.columns = ['temp_mean', 'temp_min', 'temp_max', 'rain_mean', 'rain_min', 'rain_max']
crop_conditions.reset_index(inplace=True)

In [ ]:
df1['datetime'] = pd.to_datetime(df1['datetime'])
df1['year'] = df1['datetime'].dt.year

# Agrupar por ciudad (o país si lo tienes)
climate_summary = df1.groupby('city_name')[['temperature_2m_mean', 'precipitation_sum']].mean().reset_index()


In [ ]:
def is_suitable(row, crop_data):
    return (crop_data['temp_min'] <= row['temperature_2m_mean'] <= crop_data['temp_max'] and
            crop_data['rain_min'] <= row['precipitation_sum'] <= crop_data['rain_max'])

results = []

for _, crop_row in crop_conditions.iterrows():
    crop = crop_row['Crop']
    for _, city_row in climate_summary.iterrows():
        suitable = is_suitable(city_row, crop_row)
        results.append({
            'Crop': crop,
            'City': city_row['city_name'],
            'Avg Temp': city_row['temperature_2m_mean'],
            'Avg Rainfall': city_row['precipitation_sum'],
            'Suitable': 'si' if suitable else 'no'
        })


df_results = pd.DataFrame(results)


In [ ]:
ciudades_aptas = top_promedios.index

for city in ciudades_aptas:
    print(f"\n===== {city} =====")

    df_grupo_apto = df_results[df_results['City'] == city]

    # Verifica si 'index' existe como columna, si no, no lo uses como índice
    if 'index' in df_grupo_apto.columns:
        df_pivot_apto = df_grupo_apto.set_index('index')[['Crop', 'City', 'Avg Temp', 'Avg Rainfall']]
    else:
        df_pivot_apto = df_grupo_apto[['Crop', 'Avg Temp', 'Avg Rainfall']]

    display(df_pivot_apto)  # Esto funciona en Jupyter o IPython


#**Insights**

* **Ciudades calurosas**: Hay algunas ciudades con un promedio de temperatura máxima muy alto durante todo el año (ej. Dubai, Abu Dhabi), mientras que otras tienen veranos muy calurosos, pero inviernos más suaves (ej. Miami, Phoenix).

* **Ciudades frías**: Las ciudades frías muestran una variación considerable en la temperatura a lo largo del año. Algunas tienen temperaturas bajo cero durante el invierno (ej. Toronto, Moscú).

* **Amplitud térmica**: En general, la amplitud térmica tiende a ser mayor en las ciudades calurosas, especialmente en las que están ubicadas en zonas desérticas.
* **Sensación térmica**: La diferencia entre la temperatura real y la sensación térmica es significativa en muchas ciudades, especialmente durante los meses de verano.
* **Probabilidad de lluvia**: La probabilidad de lluvia varía de forma considerable entre las ciudades y a lo largo del año.
* **Agricultura**: Existe un grupo de ciudades con condiciones agroclimáticas muy favorables (temperatura, precipitación, etc.), lo que podría ser un atractivo para la inversión en este sector.
* **Turismo**: Las ciudades con clima ideal para turismo tienen temperaturas agradables durante la mayor parte del año y pocas precipitaciones.


# Proyecto: Selección Óptima de Ubicaciones para Cultivos

## Conclusiones del Proyecto

### Ciudades con Mayor Aptitud para la Agricultura

Tras el análisis detallado de variables climáticas clave —**temperatura media anual, precipitación y radiación solar**— se identificaron las siguientes **24 ciudades** como las más aptas para el desarrollo agrícola diversificado:


In [ ]:
ciudades_optimas = [
    "Kuwait City", "Riyadh", "Miami", "Rio de Janeiro", "Hanoi", "San Diego",
    "Hong Kong", "Orlando", "Taipei", "New Orleans", "Tel Aviv", "Brisbane",
    "São Paulo", "Boston", "Athens", "Valencia", "Philadelphia", "Buenos Aires",
    "Nashville", "Sydney", "Los Angeles", "Naples", "Mexico City", "Washington"
]

for i, ciudad in enumerate(ciudades_optimas):
    print(f"{i+1}. {ciudad}")


Estas ciudades comparten condiciones climáticas **moderadamente cálidas y estables**, con:

- Temperatura media aproximada: **17.1 °C**
- Precipitación media: **3.21 mm**

Lo cual las hace ideales para cultivos de **alto valor económico y nutricional**.

---

### Principales Cultivos Recomendados

Con base en las condiciones agroclimáticas, se identificaron los siguientes cultivos como altamente compatibles con las zonas seleccionadas:



In [ ]:
import pandas as pd

cultivos = {
    "ID": [98, 198, 298, 398, 498, 598],
    "Cultivo": ["Barley", "Cotton", "Maize", "Rice", "Soybean", "Wheat"],
    "Temperatura Media (°C)": [17.106569]*6,
    "Precipitación (mm)": [3.210115]*6
}

df_cultivos = pd.DataFrame(cultivos)
df_cultivos

Estos cultivos presentan:

- Alta adaptabilidad climática
- Relevancia económica internacional
- Potencial para sistemas de rotación y producción diversificada

---

### Interpretación de Resultados

- Las ciudades seleccionadas tienen un **alto potencial agrícola** subutilizado o con posibilidades de expansión.
- La **uniformidad climática** permite planificar con precisión los ciclos agrícolas.
- La selección de cultivos incluye **granos, fibras y leguminosas**, adecuados para distintas estrategias productivas.

---

### Aplicaciones Potenciales

- Planificación estratégica de zonas agrícolas
- Implementación de sistemas de producción resilientes al clima
- Promoción de la seguridad alimentaria y reducción de riesgos
- Inversión agroindustrial en regiones urbanas y periurbanas
